In [2]:
import geopandas as gp
import pandas as pd
import numpy as np
from scipy.stats import linregress
import subprocess
import os,sys
import emcee
import statsmodels.api as sm
import pylab
from scipy.stats import norm
import seaborn as sn
import corner
from scipy import optimize,linalg
import matplotlib.pyplot as plt
import multiprocessing 

def shell(cmd):
    #print(cmd)
    process = subprocess.Popen([sys.executable]+cmd.split(" "),
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,text=True)
    stdout, stderr = process.communicate()
    if not process.returncode==0:
        try:
            print(cmd)
            print(stdout)
            print(stderr)
        except:
            print("failed to decode stdout/stderr")
        return False
    #print(stdout)
    return True



ImportError: cannot import name '_centered' from 'scipy.signal.signaltools' (C:\Users\Crispin\Anaconda2\envs\bayesiandrape\lib\site-packages\scipy\signal\signaltools.py)

In [ ]:
import numpy as np
import torch
torch.from_numpy(np.array([]))

In [ ]:
input_nets = {"a470":"../data/a470_test_prep.shp"
              }
def test_drape(input_net,z_error_prior_scale,slope_prior_scale,slope_continuity_prior_scale,pitch_angle_prior_scale,refresh=True):
    input_net_filename = input_nets[input_net]
    bayes_output_filename = "../scratch/"+os.path.basename(input_net_filename)\
                            + f"-bdrapez-{z_error_prior_scale}-{slope_prior_scale}-{slope_continuity_prior_scale}-{pitch_angle_prior_scale}.shp"
    if refresh or not os.path.exists(bayes_output_filename):
        res=shell(f"../BayesianDrape.py --TERRAIN-INPUT=../data/wider_terrain_50/all.tif --POLYLINE-INPUT={input_net_filename} "
              f"--OUTPUT={bayes_output_filename} --Z-ERROR-PRIOR-SCALE={spatial_mismatch_prior_scale} "
              f"--SLOPE-CONTINUITY-PRIOR-SCALE={slope_continuity_prior_scale} --SLOPE-PRIOR-SCALE={slope_prior_scale} "
              f"--PITCH-ANGLE-PRIOR-SCALE={pitch_angle_prior_scale} "
              f"--MAXITER=20000 --DECOUPLE-FIELD=bridge --IGNORE-PROJ-MISMATCH --SPATIAL-MISMATCH-MAX=200")
        if not res:
            return False
    return bayes_output_filename


In [ ]:
def markov_speed_factor(slope):
    '''https://link.springer.com/article/10.1007/s11116-019-10021-x#Tab5'''
    return (slope>0)*np.exp(-9.044*slope)+(slope<=0)*1
        

In [ ]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

def elev_change(geom):
    ec = 0
    for (_,_,z1),(_,_,z2) in pairwise(list(geom.coords)):
        ec += abs(z2-z1)
    return ec

def plot_elev_profile(geom,title):
    plt.figure()
    zs = [geom.coords[0][2]]
    xs = [0]
    for (x1,y1,z1),(x2,y2,z2) in  pairwise(list(geom.coords)):
        xs += [xs[-1] + ((y2-y1)**2+(x2-x1)**2)**0.5]
        zs += [z2]
    plt.scatter(xs,zs)
    plt.plot(xs,zs)
    plt.xlabel("Horizontal distance (metres)")
    plt.ylabel("Elevation (metres)")
    plt.title(title)
    plt.show()

def cycletime(slope,length):
    # bodge based on linear model for now:
    # 0% slope = factor of 1
    # 5% slope = factor of 2.2 
    # slopefac = slope*100/5*1.2+1
    slopefac = 1./markov_speed_factor(slope)
    return slopefac*length

# this is equiv to ct = ec*100/5*1.2 + len when previously we were comparing on ec
# i.e. comparing on 24*ec+len which will lessen importance of ec on long links
# is this the desired result? i wanted to study outliers more; i guess this studies outliers of grade?

def get_resids_r_ll(df,x,y):
    lr = linregress(df[x],df[y])
    resids = df[y]-(lr.intercept+lr.slope*df[x])
    scale = resids.std()
    loglik = norm(loc=0,scale=scale).logpdf(resids).sum()
    return resids,lr.rvalue,loglik,lr.slope,lr.intercept
        
   
def evaluate_drape_raw(smps,sps=90,scps=90,paps=np.inf,compare="ELEVCHANGE",maxct=8000,maxec=300,label=""):
    #slope = np.arctan(sloperat*smp)*180/np.pi did i have this before to remove interactions?
    draped_net_filename = test_drape("a470",smps,sps,scps,paps)
    if not draped_net_filename:
        return -np.inf

    draped_net = gp.read_file(draped_net_filename)
    draped_net["draped_elev_change"]=[elev_change(geom) for geom in draped_net.geometry]
    draped_net["length"]=draped_net.geometry.length

    def height(s):
        return float(s)
    draped_net["os_elev_change"] = draped_net.apply(lambda row: height(row["inDirectio"])+height(row["inOpposite"]),
                                                    axis=1)

    draped_net["os_slope"] = draped_net.os_elev_change/draped_net.length
    draped_net["draped_slope"] = draped_net.draped_elev_change/draped_net.length
    draped_net["os_cycletime"] = cycletime(draped_net.os_slope,draped_net.length)
    draped_net["draped_cycletime"] = cycletime(draped_net.draped_slope,draped_net.length)

    
    resids,r,ll,slope,intercept = get_resids_r_ll(draped_net,"draped_elev_change","os_elev_change")
    elev_error_per_len = abs(resids).sum()/draped_net.length.sum()
    #print ("Max elevation outliers",min(resids),max(resids))
    draped_net["Eresid"] = resids
    #resids,r,ll,_ = get_resids_r_ll(draped_net,"os_cycletime","draped_cycletime")
    #print ("Mean abs ct error per km",abs(resids).mean()/draped_net.length.sum()*1000)
    #print ("Max ct outliers",min(resids),max(resids))
    #draped_net["CTresid"] = resids
    #draped_net.to_file(draped_net_filename)

    #print("max ct",max(draped_net.os_cycletime.max(),draped_net.draped_cycletime.max()))
    #maxec_actual = max(draped_net.os_elev_change.max(),draped_net.draped_elev_change.max())
    #assert maxec_actual<maxec

    #draped_net.plot.scatter("os_elev_change","draped_elev_change",xlim=(0,maxec),ylim=(0,maxec))
    #draped_net.plot.scatter("os_cycletime","draped_cycletime",xlim=(0,maxct),ylim=(0,maxct))

    # ID=1172 in highways_finaltest_prep
    a470 = draped_net[draped_net.ID==-428].geometry.iloc[0]
    plot_elev_profile(a470,"Problem elevation profile 3 (a470)")
    
       
    outputs = [
        ("label",label),
        ("smp",smps),
        ("slope",sps),
        ("cont",scps),
        ("ang",paps),
        ("R",r),
        ("loglik",ll),
        ("coeff",slope),
        ("int",intercept),
        ("ec/len err",elev_error_per_len),
        ("min outlier",min(resids)),
        ("max outlier",max(resids)),
        ("a470 ec",elev_change(a470)),
    ]
    for name,value in outputs:
        print(f"{name}: {value}")
    
    keys = [x[0] for x in outputs]
    values = [x[1] for x in outputs]
    outfile = "sensitivity4.csv"
    try:
        df = pd.read_csv(outfile)
    except FileNotFoundError:
        df = pd.DataFrame(columns=keys)
    df = df.append(dict(zip(keys,values)),ignore_index=True)
    df.to_csv(outfile,index=False)


In [ ]:
zeps = np.linspace(0,5,10)

def sensitivity(slope=90,cont=np.inf,pitch=np.inf,label="sens"):
    for smp in smps:
        evaluate_drape_raw(smp,slope,cont,pitch,label=label)

In [ ]:
#sensitivity(slope=2.66,pitch=1.28,label="slp+pitch2")
#sensitivity(slope=2.66,cont=2.66,pitch=1.28*2,label="slp+cont+weakpitch2")
sensitivity(slope=2.66,label="slope2")
sensitivity(cont=2.66,label="cont2")
#sensitivity(pitch=1.28,label="pitch2")
sensitivity(slope=2.66,cont=2.66,label="slp+cont2")
#sensitivity(slope=2.66,cont=2.66,pitch=1.28,label="all2")
#sensitivity(slope=2.66,pitch=1.28*2,label="slp+weakpitch2")
#sensitivity(slope=2.66,cont=2.66/2,label="slp+weakcont")





In [ ]:
models = [("slope2","Slope"),
          ("slp+weakcont","Slope + weak continuity"),
         ("cont2","Slope continuity"),
         ("pitch2","Pitch angles"),
         ("slp+cont2","Slope + slope continuity"),
         ("slp+pitch2","Slope + pitch angles"),
         #("slp+cont+weakpitch2","Slope + continuity + weak pitch"),
         ("all2","Slope + continuity + pitch")]
         #("slp+weakpitch2","Slope + weak pitch") ]

def senstest(column,ylabel,title,legend=True):
    plt.figure()
    df = pd.read_csv("sensitivity4.csv")
    for key,label in models:
        model = df[df.label==key]
        plt.plot(model.smp,model[column],label=label)
        model = model[model.smp>3]
        maxrow = model[model[column]==model[column].max()]
        minrow = model[model[column]==model[column].min()]
        print(f"Max {column} for {key}: {float(maxrow[column]):.3f} for smp={float(maxrow.smp)}")
        print(f"Min {column} for {key}: {float(minrow[column]):.3f} for smp={float(minrow.smp)}")
    plt.xlabel("Spatial Mismatch Prior Scale (metres)")
    plt.ylabel(ylabel)
    if legend:
        plt.legend()
    plt.title(title)
    plt.show()

    
senstest("loglik","Log Likelihood","Fit with OS data")
senstest("R","R","Correlation with OS data")
senstest("coeff","Regression Coefficient","Bias")
senstest("int","metres","Regression intercept")
senstest("ec/len err","ratio","Total elevation error per unit length")
senstest("min outlier","metres","Min outlier (overpredicted elevation change)")
senstest("max outlier","metres","Max outlier (underpredicted elevation change)")
senstest("a470 ec","metres","Elevation change on a470 link")
#senstest("ncn ec","metres","Elevation change on NCN link",True)
